In [1]:
import pandas as pd
from keys import groqcloud_key
import langchain_groq as lg
ChatGroq = lg.ChatGroq

In [3]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key= groqcloud_key
)

response = llm.invoke("Who is MS Dhoni? Keep it under 50 words.")
response

AIMessage(content='MS Dhoni is a former Indian cricketer and captain of the Indian national team. He is widely regarded as one of the greatest wicket-keeper batsmen in cricket history. Dhoni led India to several major tournament victories, including the 2011 Cricket World Cup and the 2007 ICC World Twenty20.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 48, 'total_tokens': 114, 'completion_time': 0.264, 'prompt_time': 0.010146568, 'queue_time': 0.0032997819999999994, 'total_time': 0.274146568}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'stop', 'logprobs': None}, id='run-0761ea53-7636-4906-b429-5724410b57db-0', usage_metadata={'input_tokens': 48, 'output_tokens': 66, 'total_tokens': 114})

In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-42819")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Senior Data Analyst

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsSenior Data AnalystBeaverton, OregonBecome a Part of the NIKE, Inc. Team
NIKE, Inc. does mo

In [48]:
from langchain_core.prompts import PromptTemplate

prompt_extract= PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
     """
)

In [49]:
chain_extract = prompt_extract | llm
response = chain_extract.invoke(input = page_data)
print(response)

content='```json\n{\n  "role": "Senior Data Analyst",\n  "experience": "2 years of experience in the job offered or in a data-related occupation",\n  "skills": [\n    "Snowflake",\n    "AWS",\n    "Data governance",\n    "Data Analysis",\n    "Data Lake",\n    "Data Warehouse",\n    "MMX",\n    "CBD",\n    "Data Modeling",\n    "Data Profiling",\n    "Databricks",\n    "ETL tools",\n    "SQL",\n    "Python"\n  ],\n  "description": "Design, develop, and program methods, processes, and systems to consolidate and analyze structured/unstructured diverse sources to generate actionable insights and solutions for client services and product enhancement; build products for analysis; interact with product and service teams to identify questions and issues for data analysis and experiments; develop and code software programs, algorithms and automated processes to cleanse, integrate, and evaluate large datasets from multiple disparate sources; identify meaningful insights from large data and meta

In [50]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(response.content)
job = json_res

In [51]:
job

{'role': 'Senior Data Analyst',
 'experience': '2 years of experience in the job offered or in a data-related occupation',
 'skills': ['Snowflake',
  'AWS',
  'Data governance',
  'Data Analysis',
  'Data Lake',
  'Data Warehouse',
  'MMX',
  'CBD',
  'Data Modeling',
  'Data Profiling',
  'Databricks',
  'ETL tools',
  'SQL',
  'Python'],
 'description': 'Design, develop, and program methods, processes, and systems to consolidate and analyze structured/unstructured diverse sources to generate actionable insights and solutions for client services and product enhancement; build products for analysis; interact with product and service teams to identify questions and issues for data analysis and experiments; develop and code software programs, algorithms and automated processes to cleanse, integrate, and evaluate large datasets from multiple disparate sources; identify meaningful insights from large data and metadata sources; interpret and communicate insights and findings from analysis a

In [52]:
import pandas as pd

df = pd.read_csv("/Users/karananand/Cold-Email-Generator/my_portfolio.csv")
df.head(2)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio


In [53]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [54]:
# results = collection.query(
#     query_texts=['Query is about Python'],
#     n_results=2
# )
# results

In [55]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': '

In [56]:
str(job)

"{'role': 'Senior Data Analyst', 'experience': '2 years of experience in the job offered or in a data-related occupation', 'skills': ['Snowflake', 'AWS', 'Data governance', 'Data Analysis', 'Data Lake', 'Data Warehouse', 'MMX', 'CBD', 'Data Modeling', 'Data Profiling', 'Databricks', 'ETL tools', 'SQL', 'Python'], 'description': 'Design, develop, and program methods, processes, and systems to consolidate and analyze structured/unstructured diverse sources to generate actionable insights and solutions for client services and product enhancement; build products for analysis; interact with product and service teams to identify questions and issues for data analysis and experiments; develop and code software programs, algorithms and automated processes to cleanse, integrate, and evaluate large datasets from multiple disparate sources; identify meaningful insights from large data and metadata sources; interpret and communicate insights and findings from analysis and experiments to product, s

In [57]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Analysis Solutions for Your Business Needs

Dear Hiring Manager,

I came across the Senior Data Analyst job description at your esteemed organization and was impressed by the role's focus on leveraging data analysis to drive business growth and product enhancement. As a Business Development Executive at AtliQ, I'd like to introduce our company's capabilities in providing tailored data analysis solutions that align with your requirements.

AtliQ is an AI & Software Consulting company dedicated to facilitating the seamless integration of business processes through automated tools. Our team of experts has extensive experience in designing, developing, and programming methods, processes, and systems to consolidate and analyze structured/unstructured diverse sources. We've empowered numerous enterprises with scalable, optimized, and cost-effective solutions, resulting in heightened overall efficiency.

Our expertise in data analysis, data governance, data modeling, and 